#Loading The Model

In [0]:
from keras import models
import numpy as np
from sklearn.metrics import recall_score,precision_score,f1_score

In [0]:
path = 'drive/My Drive/Honeywell//'
model_path = path + 'model_bidir0_935.h5'

In [0]:
model = models.load_model(model_path)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 1000, 80)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1000, 160)         12960     
_________________________________________________________________
batch_normalization_22 (Batc (None, 1000, 160)         640       
_________________________________________________________________
activation_8 (Activation)    (None, 1000, 160)         0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 1000, 256)         295936    
_________________________________________________________________
batch_normalization_23 (Batc (None, 1000, 256)         1024      
_________________________________________________________________
bidirectional_8 (Bidirection (None, 1000, 256)         394240    
__________

#Prediction using the model

In [0]:
def get_y_pred(path,out1):
  X = np.load(path)
  sz = (X.shape[0]//1000)
  X = X[:sz * 1000,:]
  X = np.reshape(X,(sz,1000,80))
  
  y_pred = model.predict(X[:,:])
  
  y_pred = (y_pred>=0.5)*1
  y_pred = y_pred.flatten()
  
  
#   y_pred = np.array(y_pred,dtype=np.int8)
  np.savetxt(out1,y_pred,fmt="%d",newline=' ')
  return y_pred

In [0]:
def y_predToOut2(y_pred,out2):
  s = y_pred
  #fs = open(pathing + 'required_'+str(x)+'.txt','w+')
  #print(s[0:100])
  prev = ""
  cur = s[0]
  i = 1
  flag = not(s[0])
  start = 0
  f = open(out2, "w")
  print(len(s))
  while(i!=len(s)):
    prev = cur
    cur = s[i]
    if(prev!=cur):
      if flag == 0:
        flag = 1
        #print(str(float(start)/100) + " "+ str(float(i)/100) + " S")
        f.write(str(float(start)/100) + " "+ str(float(i)/100) + " S\n")
      else:
        #print(str(float(start)/100) + " "+ str(float(i)/100) + " NS")
        f.write(str(float(start)/100) + " "+ str(float(i)/100) + " NS\n")
        flag = 0
      start  = i
    
    i+=1
  if flag == 0:
    flag = 1
    #print(str(float(start)/100) + " "+ str(float(i)/100) + " S")
    f.write(str(float(start)/100) + " "+ str(float(i)/100) + " S\n")
  else:
    #print(str(float(start)/100) + " "+ str(float(i)/100) + " NS")
    f.write(str(float(start)/100) + " "+ str(float(i)/100) + " NS\n")
    flag = 0

In [0]:
def predict(inp,out1,out2):
  y_pred = get_y_pred(inp,out1)
  y_predToOut2(y_pred,out2)

In [0]:
import os
os.listdir()

['.config', 'outputSnS.txt', 'drive', 'output1_0.txt', 'sample_data']

In [0]:
#inp = '/Edu/Hackathons/Honeywell/.npy'
inp = 'drive/My Drive/newtest.npy'
out1 = 'output1_0.txt'
out2 = 'outputSnS.txt'
predict(inp,out1,out2)

146
(1, 146, 1)
6146


# Evaluation of Model

In [0]:
# Load preprocessed training examples
X = np.load(path + "X_data.npy")
Y = np.load(path + 'Y_data.npy')
shape_x = (1800,1000,80)
X1 = np.reshape(X,shape_x)
shape_y = (1800,1000,1)
Y1 = np.reshape(Y,shape_y)

In [0]:
# Load preprocessed dev set examples
X_dev1 = np.load("newtest.npy")
Y_dev1 = np.load('newtest_labels.npy')

shape_x = (6,1000,80)
shape_y = (6,1000,1)

X_dev1 = np.reshape(X_dev1[:6000,:],shape_x)
Y_dev1 = np.reshape(Y_dev1[:6000,:],shape_y)
print(X_dev1.shape)
Y_dev1.shape

(6, 1000, 80)


(6, 1000, 1)

In [0]:
# Load preprocessed dev set examples
X_dev2 = np.load(path + "X_dev2.npy")
Y_dev2 = np.load(path + 'Y_dev2.npy')

shape_x = (54,1000,80)
shape_y = (54,1000,1)

X_dev2 = np.reshape(X_dev2[:54000,:],shape_x)
Y_dev2 = np.reshape(Y_dev2[:54000,:],shape_y)
print(X_dev2.shape)
Y_dev2.shape

(54, 1000, 80)


(54, 1000, 1)

In [0]:
loss, acc = model.evaluate(X1[:,:,:], Y1[:,:,:])
print("Train set accuracy = ", acc)

y_pred = model.predict(X1[:,:])
y_true = Y1.flatten()

y_pred = (y_pred>=0.5)*1
y_pred = y_pred.flatten()
print('Precision Score = ',precision_score(y_true, y_pred))
print('Recall Score = ',recall_score(y_true, y_pred))
print('F1 Score = ',f1_score(y_true, y_pred))

1800/1800 [==============================] - 72s 40ms/step
Train set accuracy =  0.9399661127726237
Precision Score =  0.9285696738714647
Recall Score =  0.8870944760059207
F1 Score =  0.9073583661754001


In [0]:
loss, acc = model.evaluate(X_dev1[:,:,:], Y_dev1[:,:,:])
print("Dev set accuracy = ", acc)

y_pred = model.predict(X_dev1[:,:])
y_pred = (y_pred>=0.5)*1
y_pred = y_pred.flatten()
y_true = Y_dev1.flatten()
print('Precision Score = ',precision_score(y_true, y_pred))
print('Recall Score = ',recall_score(y_true, y_pred))
print('F1 Score = ',f1_score(y_true, y_pred))

6/6 [==============================] - 1s 208ms/step
Dev set accuracy =  0.9163333177566528
Precision Score =  0.8019846639603068
Recall Score =  0.9657794676806084
F1 Score =  0.8762937407589947


In [0]:
loss, acc = model.evaluate(X_dev2[:,:,:], Y_dev2[:,:,:])
print("Dev set accuracy = ", acc)

y_pred = model.predict(X_dev2[:,:])
y_true = Y_dev2.flatten()

y_pred = (y_pred>=0.5)*1
y_pred = y_pred.flatten()
print('Precision Score = ',precision_score(y_true, y_pred))
print('Recall Score = ',recall_score(y_true, y_pred))
print('F1 Score = ',f1_score(y_true, y_pred))

54/54 [==============================] - 2s 38ms/step
Dev set accuracy =  0.8891481536406057
Precision Score =  0.8701412632715494
Recall Score =  0.9945238584944467
F1 Score =  0.9281840867645648
